In [57]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"

open Octokit
open NodaTime
open NodaTime.Extensions
open XPlot.Plotly

yielding source C:\Users\kevinr\AppData\Local\Temp\nuget\33508\Project.fsproj.fsx


In [58]:
let organization = "dotnet"
let repositoryName = "try"
let options = ApiOptions()
let gitHubClient = GitHubClient(ProductHeaderValue("notebook"))

In [59]:
//var tokenAuth = new Credentials("YOUR TOKEN");
//gitHubClient.Credentials = tokenAuth;

In [60]:
let today = SystemClock.Instance.InUtc().GetCurrentDate()
let startOfTheMonth = today.With(DateAdjusters.Month(1))
let startOfTheYear = LocalDate(today.Year, 1, 1).AtMidnight()

let since t = Nullable(DateTimeOffset(t))
let await f = (async { return! f() |> Async.AwaitTask } |> Async.StartAsTask)

In [61]:
let createdIssuesRequest =
    RepositoryIssueRequest(
        Since = since (startOfTheMonth.ToDateTimeUnspecified()),
        Filter = IssueFilter.Created)
let closedIssuesRequest =
    new RepositoryIssueRequest(
                Since = since (startOfTheMonth.ToDateTimeUnspecified()),
                State = ItemStateFilter.Closed)
let thisYearIssuesRequest =
    RepositoryIssueRequest(
        Since = since (startOfTheYear.ToDateTimeUnspecified()),
        State = ItemStateFilter.All)

In [62]:
let createdThisMonth = await (fun () -> gitHubClient.Issue.GetAllForRepository(organization, repositoryName, createdIssuesRequest) )
let closedThisMonth =  await (fun () -> gitHubClient.Issue.GetAllForRepository(organization, repositoryName, closedIssuesRequest) )
let thisYearIssues =   await (fun () -> gitHubClient.Issue.GetAllForRepository(organization, repositoryName, thisYearIssuesRequest) )

In [63]:
let openSoFar =
        createdThisMonth
            .Result
            .OrderBy(fun i -> i.CreatedAt)
            .Where(fun i -> i.State.StringValue = "open")
            .ToArray()

let openByMonthOfCreation =
        openSoFar
            .GroupBy(fun i -> {| Year = i.CreatedAt.Year; Month = i.CreatedAt.Month |})
            .Select(fun g ->  {| Date = g.Key; Count = g.Count() |})
            .ToArray()

let closedSoFar =
        thisYearIssues
            .Result
            .OrderBy(fun i -> i.CreatedAt)
            .Where(fun i -> i.State.StringValue = "closed")
            .ToArray()

let closedByMonthOfClosure =
        closedSoFar
            .GroupBy(fun i -> {| Year = i.ClosedAt.Value.Year; Month = i.ClosedAt.Value.Month |})
            .Select(fun g ->  {| Date = g.Key; Count = g.Count() |})
            .ToArray();

let mutable totalOpenIssues = thisYearIssues.Result.Count

let openCountByMonth =
    closedSoFar
        .GroupBy(fun i -> {| Year = i.CreatedAt.Year; Month = i.CreatedAt.Month |})
        .Select(fun g ->
                    let count = g.Count()
                    let dataPoint = {| Date = g.Key; Count = totalOpenIssues |}
                    totalOpenIssues <- totalOpenIssues - count
                    dataPoint)
        .ToArray()

In [64]:
let createdThisMonthByDay = 
    createdThisMonth.Result.GroupBy(fun i -> DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day))
    
let lineChart =
    Chart.Line(createdThisMonthByDay.Select(fun g -> Tuple<DateTime,int>(g.Key, g.Count())))

lineChart.WithTitle("Daily Issues")

display(lineChart)

In [65]:
let table =
    openSoFar
        .Select(fun i -> {| CreatedAt = i.CreatedAt; Title = i.Title; State = i.State.StringValue; Number = i.Number |})
        .OrderByDescending(fun d -> d.CreatedAt)

display(table)

index,CreatedAt,Number,State,Title
0,2019-11-01 13:28:34Z,588,open,added content
1,2019-11-01 04:52:29Z,586,open,[WIP] #r nuget improvements
2,2019-10-31 21:29:54Z,585,open,Use LaTeX formatter when string has the LaTeX pattern
3,2019-10-31 16:43:40Z,584,open,#r is unable to load an assembly that's already on disk
4,2019-10-31 15:11:39Z,583,open,Improve package load error formatting
5,2019-10-31 14:59:43Z,582,open,#r nuget doesn't understand wildcard package versions in C# kernel
6,2019-10-31 13:30:35Z,581,open,upgrade asp.net packages
7,2019-10-30 17:37:29Z,577,open,Change magic command parsing and syntax to not use `%` prefix
8,2019-10-29 22:55:15Z,575,open,Consider adapting Fable.React for F# formatting of HTML
9,2019-10-28 20:52:54Z,570,open,Custom formatter for F# discriminated unions


In [66]:
let lineChart =
    Chart.Line(openByMonthOfCreation.Select(fun g -> Tuple<DateTime, int>(DateTime(g.Date.Year, g.Date.Month, 1), g.Count)))

lineChart.WithTitle("Issues still opened grouped by month")

display(lineChart)

In [67]:
let idleByMonth =
    openSoFar
        .Where(fun i -> i.Comments = 0)
        .GroupBy(fun i -> DateTime(i.CreatedAt.Year, i.CreatedAt.Month, 1))
        .Select(fun g -> {| Date = g.Key; Count = g.Count() |})
        .ToArray()

let activeByMonth =
    openSoFar
        .Where(fun i -> i.Comments > 0)
        .GroupBy(fun i -> DateTime(i.CreatedAt.Year, i.CreatedAt.Month, 1))
        .Select(fun g -> {| Date = g.Key; Count = g.Count() |})
        .ToArray()

let idleSeries =
    Graph.Scattergl(
        name = "Idle",
        y = idleByMonth.Select(fun g -> g.Count).ToArray(),
        x = idleByMonth.Select(fun g -> g.Date).ToArray())

let activeSeries =
    Graph.Scattergl(
        name = "Active",
        y = activeByMonth.Select(fun g -> g.Count).ToArray(),
        x = activeByMonth.Select(fun g -> g.Date ).ToArray())

let chart = Chart.Plot([|idleSeries; activeSeries|])

chart.WithTitle("Idle and active open issue report")
display(chart);

In [68]:
let openDataPoints =
    openByMonthOfCreation
        .Select(fun g -> {| y = g.Count; x = DateTime(g.Date.Year, g.Date.Month, 1) |})
        .OrderBy(fun d -> d.x)
        .ToArray()

let closedDataPoints =
    closedByMonthOfClosure
        .Select(fun g -> {| y = g.Count; x = DateTime(g.Date.Year, g.Date.Month, 1) |})
        .OrderBy(fun d -> d.x)
        .ToArray()

let openCountByMonthDataPoints =
    openCountByMonth
        .Select(fun g -> {| y = g.Count; x = DateTime(g.Date.Year, g.Date.Month, 1) |})
        .OrderBy(fun d -> d.x)
        .ToArray()

let openSeries =
    Graph.Scattergl(
        name = "Created",
        y = openDataPoints.Select(fun g -> g.y).ToArray(),
        x = openDataPoints.Select(fun g -> g.x).ToArray())

let closeSeries =
    Graph.Scattergl(
        name = "Closed",
        y = closedDataPoints.Select(fun g -> g.y ).ToArray(),
        x = closedDataPoints.Select(fun g -> g.x ).ToArray())

let stillOpenSeries =
    Graph.Scattergl(
        name = "Open",
        y = openCountByMonthDataPoints.Select(fun g -> g.y ).ToArray(),
        x = openCountByMonthDataPoints.Select(fun g -> g.x ).ToArray())

let chart =
    Chart.Plot([| openSeries; closeSeries; stillOpenSeries |])

chart.WithTitle("Issue report for the current year")
display(chart)

In [69]:
let forks = await (fun () -> gitHubClient.Repository.Forks.GetAll(organization, repositoryName))

let forkCreateByMonth =
    forks.Result
         .GroupBy(fun f -> DateTime(f.CreatedAt.Year, f.CreatedAt.Month,  f.CreatedAt.Day))
         .Select(fun g -> {| Date = g.Key; Count = g.Count() |})
         .OrderBy(fun g -> g.Date)
         .ToArray()

let forkLastUpdateByMonth =
    forks.Result
         .GroupBy(fun f -> DateTime(f.UpdatedAt.Year, f.UpdatedAt.Month,  f.UpdatedAt.Day))
         .Select(fun g -> {| Date = g.Key; Count = g.Count() |})
         .OrderBy(fun g -> g.Date)
         .ToArray()

let total = 0
let forkCountByMonth =
    forkCreateByMonth
        .OrderBy(fun g -> g.Date)
        .Select(fun g -> {| Date = g.Date; Count = total + g.Count |})
        .ToArray()

let forkCreationSeries =
    Graph.Scattergl(
        name = "created by month",
        y = forkCreateByMonth.Select(fun g -> g.Count).ToArray(),
        x = forkCreateByMonth.Select(fun g -> g.Date).ToArray())

let forkTotalSeries =
    Graph.Scattergl(
        name = "running total",
        y = forkCountByMonth.Select(fun g -> g.Count ).ToArray(),
        x = forkCountByMonth.Select(fun g -> g.Date ).ToArray())

let forkUpdateSeries =
    Graph.Scattergl(
        name = "last update by month",
        y = forkLastUpdateByMonth.Select(fun g -> g.Count ).ToArray(),
        x = forkLastUpdateByMonth.Select(fun g -> g.Date ).ToArray())

let chart =
    Chart.Plot([| forkCreationSeries; forkTotalSeries; forkUpdateSeries |])

chart.WithTitle("Fork activity")
display(chart)